In [70]:
#Packages and modules
#Get the required modules/packages
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import re
from langchain.schema import Document 
import os
from dotenv import load_dotenv
from langchain_text_splitters import TextSplitter
from docx import Document

In [ ]:
import os
# Access the API key from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

#change

In [15]:
# Load the vectorstore

persist_directory = "chroma_storage"

embedding_model = OpenAIEmbeddings()

vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory=persist_directory
)

print("Vector store loaded from disk.")

Vector store loaded from disk.


## Delete vectorstore (if needed)

In [ ]:
# Delete the existing vectorstore
import shutil

# Specify the directory you used for persistent storage
persist_directory = "chroma_storage"

# Delete the directory and its contents
shutil.rmtree(persist_directory)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'chroma_storage\\chroma.sqlite3'

# Define the data sources

### Interview

Here the interview transcript is prepared to be embedded in the vector store

1. The full text is grabbed from the document
2. Metadata such as speaker name, role, and bio are defined. But also metadata
    such as type, source, data, etc.
3. LangChainDocument is prepared
4. Splitter is defined (chunk size + overlap)
5. for loop splits the document into chunks and attaches relevant metadata
6. The chunks are appended in an empty list named "splits"
7. The chunks (or splits) are embedded into the vectorstore Chroma


#### Interview without metadata

In [ ]:
""" from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document as LangChainDocument
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Load the Word document and extract text
doc = Document("corrected_interview.docx")
interview_text = "\n".join([para.text for para in doc.paragraphs])

# Define speaker metadata
speaker_info = {
    "Lex Fridman": {
        "role": "Interviewer",
        "bio": "Lex Fridman is a Russian-American computer scientist and podcaster, known for his work in artificial intelligence and his podcast featuring interviews with prominent figures across various fields."
    },
    "Roman Yampolskiy": {
        "role": "Guest",
        "bio": "Roman Yampolskiy is a Latvian computer scientist specializing in AI safety and cybersecurity, holding a Ph.D. from the University at Buffalo, and currently directing the Cyber Security Lab at the University of Louisville's Speed School of Engineering."
    }
}

# Identify speaker based on keywords
def identify_speaker(text):
    if "Lex Fridman" in text:
        return "Lex Fridman"
    elif "Roman Yampolskiy" in text:
        return "Roman Yampolskiy"
    return "Unknown"

# Create the initial LangChain document with the entire text and minimal metadata
documents = [LangChainDocument(page_content=interview_text, metadata={"source": "interview"})]

# Initialize RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the document into chunks with metadata
splits = []
for chunk in splitter.split_documents(documents):
    speaker = identify_speaker(chunk.page_content)
    
    # Add detailed metadata for each chunk
    chunk.metadata.update({
        "speaker": speaker,
        "speaker_role": speaker_info.get(speaker, {}).get("role", "Unknown"),
        "speaker_bio": speaker_info.get(speaker, {}).get("bio", "No bio available")
    })
    splits.append(chunk)

# Verify the splits and metadata
for doc in splits:
    print("Content:", doc.page_content[:200], "...")  # Print first 200 characters for brevity
    print("Metadata:", doc.metadata)
    print("-" * 50)

# Embed the chunks into Chroma vectorstore
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory = persist_directory)

 """

Content: Lex Fridman: What to you is the probability that superintelligent AI will destroy all human civilization?

Roman Yampolskiy: What's the timeframe?

Lex Fridman: Let's say a hundred years, in the next  ...
Metadata: {'source': 'interview', 'speaker': 'Lex Fridman', 'speaker_role': 'Interviewer', 'speaker_bio': 'Lex Fridman is a Russian-American computer scientist and podcaster, known for his work in artificial intelligence and his podcast featuring interviews with prominent figures across various fields.'}
--------------------------------------------------
Content: Roman Yampolskiy: So the problem of controlling AGI or superintelligence, in my opinion, is like the problem of creating a perpetual safety machine by analogy with a perpetual motion machine. It's imp ...
Metadata: {'source': 'interview', 'speaker': 'Roman Yampolskiy', 'speaker_role': 'Guest', 'speaker_bio': "Roman Yampolskiy is a Latvian computer scientist specializing in AI safety and cybersecurity, holding a Ph.D.

#### Code for interview embedding interview with updated metadata

In [7]:
from docx import Document as DocxDocument
from langchain.schema import Document as LangChainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load the Word document and extract text
doc = DocxDocument("corrected_interview.docx")
interview_text = "\n".join([para.text for para in doc.paragraphs])

# Define speaker metadata
speaker_info = {
    "Lex Fridman": {
        "role": "Interviewer",
        "bio": "Lex Fridman is a Russian-American computer scientist and podcaster, known for his work in artificial intelligence and his podcast featuring interviews with prominent figures across various fields."
    },
    "Roman Yampolskiy": {
        "role": "Guest",
        "bio": "Roman Yampolskiy is a Latvian computer scientist specializing in AI safety and cybersecurity, holding a Ph.D. from the University at Buffalo, and currently directing the Cyber Security Lab at the University of Louisville's Speed School of Engineering."
    }
}

# Function to identify speaker based on keywords in the text
def identify_speaker(text):
    if "Lex Fridman" in text:
        return "Lex Fridman"
    elif "Roman Yampolskiy" in text:
        return "Roman Yampolskiy"
    return "Unknown"

# Initialize Document with minimal metadata
documents = [LangChainDocument(page_content=interview_text, metadata={"source": "YouTube"})]

# Initialize RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the document into chunks and add metadata
splits = []
for chunk in splitter.split_documents(documents):
    # Identify the speaker for each chunk
    speaker = identify_speaker(chunk.page_content)
    
    # Add detailed metadata for each chunk
    chunk.metadata.update({
        "source": "YouTube",
        "title": "P(doom): Probability that AI will destroy human civilization | Roman Yampolskiy and Lex Fridman",
        "type": "interview",
        "date": "June 13, 2024",
        "summary": (
            """In a discussion between Lex Fridman and Roman Yampolskiy, Yampolskiy expresses skepticism "
            about the safety of superintelligent AI, likening its control to the impossible creation
            of a perpetual motion machine, emphasizing the existential risks that come from potential
            mistakes in AI development. He discusses various risks associated with advanced AI, including
            existential risk (total human extinction), suffering risk (mass suffering), and Ikigai risk
            (loss of purpose), highlighting that AI could creatively find ways to harm humanity that we
            may not anticipate. Furthermore, Yampolskiy suggests that while there might be short-term
            methods to manage risks, the rapid advancement of AI capabilities may eventually lead us to
            a point where we cannot defend against its potential threats, leading him to conclude that the
            best course of action might be to avoid creating superintelligent systems altogether."""
        ),
        "speaker": speaker,
        "speaker_role": speaker_info.get(speaker, {}).get("role", "Unknown"),
        "speaker_bio": speaker_info.get(speaker, {}).get("bio", "No bio available")
    })
    splits.append(chunk)

# Verify the splits and metadata
for doc in splits:
    print("Content:", doc.page_content[:200], "...")
    print("Metadata:", doc.metadata)
    print("-" * 50)

# Embed the chunks into Chroma vectorstore
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory=persist_directory)


Content: Lex Fridman: What to you is the probability that superintelligent AI will destroy all human civilization?

Roman Yampolskiy: What's the timeframe?

Lex Fridman: Let's say a hundred years, in the next  ...
Metadata: {'source': 'YouTube', 'title': 'P(doom): Probability that AI will destroy human civilization | Roman Yampolskiy and Lex Fridman', 'type': 'interview', 'date': 'June 13, 2024', 'summary': 'In a discussion between Lex Fridman and Roman Yampolskiy, Yampolskiy expresses skepticism "\n            about the safety of superintelligent AI, likening its control to the impossible creation\n            of a perpetual motion machine, emphasizing the existential risks that come from potential\n            mistakes in AI development. He discusses various risks associated with advanced AI, including\n            existential risk (total human extinction), suffering risk (mass suffering), and Ikigai risk\n            (loss of purpose), highlighting that AI could creatively find ways

## Research report

In [8]:
# Install required packages
%pip install -qU pypdf
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("AGI Research article.pdf")

pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [10]:
# Check content of pages
print(f"{pages[0].metadata}\n")
print(pages[2].page_content)


{'source': 'AGI Research article.pdf', 'page': 0}

& Hutter, 2006 ), and the ability to achieve complex goals in complex environments (Goertzel, 2006 ). 
Whilst current ANI systems have typically been used as tools to support human behaviours, an AGI 
system would be an autonomous agent that can learn in an unsupervised manner (Firt, 2020 ; Torres, 
2019 ). Whilst, AGI does not currently exist, it is expected to arrive sometime this century (Müller & 
Bostrom, 2016 ).
Although ANI systems such as Uber’s automated vehicles can create safety risks (Stanton et al., 
2019 ), they do not, at present, pose a significant threat to humanity (Bentley, 2018 ). This is not the 
case with AGI, with many scholars discussing potential existential threats (Salmon et al., 2021 ). The 
risks associated with AGI are generated by the challenge of controlling an agent that is substantially 
more intelligent than us (Baum, 2017 ). The exponential rate at which technology is advancing, such 
as in the areas

In [11]:
import asyncio
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document as LangChainDocument
import nest_asyncio

# Apply nest_asyncio to handle async in Jupyter or similar environments
nest_asyncio.apply()

# Step 1: Load the PDF Document
file_path = "AGI Research article.pdf" # Path to the PDF file
loader = PyPDFLoader(file_path)

# Asynchronously load pages
pages = []
async def load_pdf_pages():
    async for page in loader.alazy_load():
        pages.append(page)

# Run the async function to load pages
asyncio.run(load_pdf_pages())

# Step 2: Define General Metadata for the Research Paper
metadata = {
    "type": "research paper",  # Type of document
    "title": "The risks associated with Artificial General Intelligence: A systematic review",  #The title
    "source": "Journal of Experimental & Theoretical Artificial Intelligence",  # Journal name
    "author(s)": ", ".join([  # Convert list of authors to a comma-separated string
        "Scott McLean", "Gemma J. M. Read", "Jason Thompson", 
        "Chris Baber", "Neville A. Stanton", "Paul M. Salmon"
    ]),
    "abstract": """Artificial General intelligence (AGI) offers enormous benefits for humanity,
    yet it also poses great risk. The aim of this systematic review was to
    summarise the peer reviewed literature on the risks associated with AGI.
    The review followed the Preferred Reporting Items for Systematic Reviews
    and Meta-Analyses (PRISMA) guidelines. Sixteen articles were deemed
    eligible for inclusion. Article types included in the review were classified
    as philosophical discussions, applications of modelling techniques, and
    assessment of current frameworks and processes in relation to AGI. The
    review identified a range of risks associated with AGI, including AGI
    removing itself from the control of human owners/managers, being
    given or developing unsafe goals, development of unsafe AGI, AGIs with
    poor ethics, morals and values; inadequate management of AGI, and
    existential risks. Several limitations of the AGI literature base were also
    identified, including a limited number of peer reviewed articles and
    modelling techniques focused on AGI risk, a lack of specific risk research
    in which domains that AGI may be implemented, a lack of specific definitions of the AGI functionality, and a lack of standardised AGI terminology.
    Recommendations to address the identified issues with AGI risk research
    are required to guide AGI design, implementation, and management""",  # Abstract
    "date": "August 2021",  # Publication date
    "keywords": ", ".join(["AGI", "risk assessment", "AI safety", "Artificial Intelligence"]),  # Keywords
}

# Step 3: Initialize the Text Splitter for Chunking
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
documents_with_metadata = []

# Step 4: Split each page's content and add metadata
for page in pages:
    page_number = page.metadata.get("page", "Unknown")  # Extract page number
    
    # Split text on each page into chunks
    page_chunks = splitter.split_text(page.page_content)
    
    for chunk in page_chunks:
        # Add metadata for each chunk
        chunk_metadata = {
            **metadata,  # General metadata for the document
            "page": page_number  # Specific page number metadata
        }
        
        # Create a LangChain Document for each chunk with metadata
        langchain_doc = LangChainDocument(page_content=chunk, metadata=chunk_metadata)
        documents_with_metadata.append(langchain_doc)

# Step 5: Embed the Chunks in Chroma Vector Store
vectorstore = Chroma.from_documents(documents=documents_with_metadata, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory=persist_directory)
print("Successfully embedded the chunks in Chroma Vector Store.")

Successfully embedded the chunks in Chroma Vector Store.


# Test Vector Store

In [35]:
# Sample query for retrieval (adjust the query based on your use case)
sample_query = "Will AGI pose existential risks to humanity?"

retrieved_docs = vectorstore.max_marginal_relevance_search(
    query=sample_query,
    num_results=5,
    fetch_k=15,
    filter={"type": "interview"}
)

# Inspect the retrieved documents and their metadata
print("Retrieved Documents and Metadata:")
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i + 1}:")
    print("Content:", doc.page_content)  # Show first 200 characters for context
    print("Metadata:", doc.metadata)  # Print all metadata fields
    print("-" * 50)

Retrieved Documents and Metadata:

Document 1:
Content: Roman Yampolskiy: So the problem of controlling AGI or superintelligence, in my opinion, is like the problem of creating a perpetual safety machine by analogy with a perpetual motion machine. It's impossible. Yeah, we may succeed and do a good job with GPT-5, 6, 7, but they just keep improving, learning, eventually self-modifying, interacting with the environment, interacting with malevolent actors. The difference between cybersecurity, narrow AI safety, and safety for general AI or superintelligence is that we don't get a second chance with cybersecurity. Somebody hacks your account; what's the big deal? You get a new password, a new credit card, you move on. Here, if we're talking about existential risks, you only get one chance. So you're really asking me what are the chances that we'll create the most complex software ever on the first try with zero bugs? And it will continue to have zero bugs for 100 years or more.
Metadata: 

# Wikipedia docs

In [14]:
%pip install -qU langchain_community wikipedia

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Artificial General Intelligence", load_max_docs=2).load()
len(docs)

2

In [16]:
docs[1].metadata  # metadata of the first document

{'title': 'Artificial intelligence',
 'summary': 'Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nSome high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being call

In [17]:
docs[0].page_content[:400]  # a part of the page content

'Artificial general intelligence (AGI) is a type of artificial intelligence (AI) that matches or surpasses human cognitive capabilities across a wide range of cognitive tasks. This contrasts with narrow AI, which is limited to specific tasks. Artificial superintelligence (ASI), on the other hand, refers to AGI that greatly exceeds human cognitive capabilities. AGI is considered one of the definitio'

## Chunk and embed

# News Articles

## Using WebBaseLoader

In [18]:
from bs4 import BeautifulSoup

#Get the URL of the article
url = "https://time.com/7093792/ai-artificial-general-intelligence-risks/"
loader = WebBaseLoader([url])
documents = loader.load()



#Define chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

#Split the document into chunks using the textsplitter
splits = text_splitter.split_documents(documents)

#Add metadata to each chunk
metadata = {
    "title": "Silicon Valley Takes Artificial General Intelligence Seriously—Washington Must Too",  #The title
    "journal": "Time Magazine",  # Journal name
    "authors": "Daniel Colson",  # Authors
    "summary": """highlights the urgent need for U.S. policymakers to address the rapid 
    advancements in Artificial General Intelligence (AGI). During a Senate 
    Judiciary Subcommittee hearing on September 17, 2024, whistleblowers from
    leading AI companies, including OpenAI and Google, emphasized that AGI 
    is no longer a distant speculation but an impending reality. Helen Toner,
    a former OpenAI board member, testified that companies like OpenAI,
    Google, and Anthropic are earnestly pursuing AGI development. Former
    OpenAI researcher William Saunders noted that these companies are 
    investing billions toward this goal. Senators Josh Hawley and 
    Richard Blumenthal expressed concern over the lack of regulatory 
    oversight, warning of potential risks such as cyberattacks, 
    economic upheaval, and even human extinction. The article 
    calls for immediate regulatory action, public engagement, 
    and the implementation of transparency and security mandates 
    to manage the societal impacts of AGI responsibly.""",  # Summary
    "date": "October 18th 2021",  # Publication date
    
}

# Initialize the list to store the documents with metadata
documents_with_metadata = []



In [19]:
# Take a look at splits
for doc in splits:
    print("Content:", doc.page_content[:], "...")  # Print first 200 characters for brevity
    print("Metadata:", doc.metadata)
    print("-" * 50)

Content: Silicon Valley Takes AGI Seriously—Washington Should Too | TIMESign Up for Our Ideas Newsletter POVSubscribeSubscribeSectionsHomeU.S.PoliticsWorldHealthClimateFuture of Work by CharterBusinessTechEntertainmentIdeasScienceHistorySportsMagazineTIME 2030Next Generation LeadersTIME100 Leadership SeriesTIME StudiosVideoTIME100 TalksTIMEPiecesThe TIME VaultTIME for HealthTIME for KidsTIME EdgeTIMECO2Red Border: Branded Content by TIMECouponsPersonal Finance by TIME StampedShopping by TIME StampedJoin UsNewslettersSubscribeGive a GiftShop the TIME StoreTIME Cover StoreDigital MagazineCustomer CareUS & CanadaGlobal Help CenterReach OutCareersPress RoomContact the EditorsMedia KitReprints and PermissionsMoreAbout UsPrivacy PolicyYour Privacy RightsTerms of UseModern Slavery StatementSite MapConnect with UsPresented ByIdeasAI in FocusSilicon Valley Takes Artificial General Intelligence Seriously—Washington Must TooSilicon Valley Takes Artificial General Intelligence Seriously—Washington

In [20]:
# print the first 1000 characters of article body
documents[0].page_content



"Silicon Valley Takes AGI Seriously—Washington Should Too | TIMESign Up for Our Ideas Newsletter POVSubscribeSubscribeSectionsHomeU.S.PoliticsWorldHealthClimateFuture of Work by CharterBusinessTechEntertainmentIdeasScienceHistorySportsMagazineTIME 2030Next Generation LeadersTIME100 Leadership SeriesTIME StudiosVideoTIME100 TalksTIMEPiecesThe TIME VaultTIME for HealthTIME for KidsTIME EdgeTIMECO2Red Border: Branded Content by TIMECouponsPersonal Finance by TIME StampedShopping by TIME StampedJoin UsNewslettersSubscribeGive a GiftShop the TIME StoreTIME Cover StoreDigital MagazineCustomer CareUS & CanadaGlobal Help CenterReach OutCareersPress RoomContact the EditorsMedia KitReprints and PermissionsMoreAbout UsPrivacy PolicyYour Privacy RightsTerms of UseModern Slavery StatementSite MapConnect with UsPresented ByIdeasAI in FocusSilicon Valley Takes Artificial General Intelligence Seriously—Washington Must TooSilicon Valley Takes Artificial General Intelligence Seriously—Washington Must To

## Using BeautifulSoup

### Article #1 (Time)

In [21]:
import requests
from bs4 import BeautifulSoup

# Step 1: Define the URL and fetch HTML content
url = "https://time.com/7093792/ai-artificial-general-intelligence-risks/"
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Step 2: Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Locate the main article body
# Look for an element that contains the main article text
# You may need to adjust these selectors based on the HTML structure of the Time website

article_body = soup.find("div", id="article-body-main")  # Adjust if needed
if not article_body:
    article_body = soup.find("article")  # Use <article> tag as a fallback

# Step 4: Remove unwanted elements, such as ads or related links
for unwanted in article_body.find_all(["aside", "footer", "nav", "script", "style", "figure", "div"]):
    unwanted.decompose()  # This removes the unwanted tags and their content

# Step 5: Extract and clean the text
body_text = article_body.get_text()



In [22]:
# Now we need to chunk the text into smaller parts and add metadata

# Define the metadata
metadata = {
    "type": "article",
    "title": "Silicon Valley Takes Artificial General Intelligence Seriously—Washington Must Too",
    "source": "Time Magazine",
    "author(s)": "Daniel Colson",
    "date": "October 18 2024",
    "summary": """highlights the urgent need for U.S. policymakers to address the rapid 
    advancements in Artificial General Intelligence (AGI). During a Senate 
    Judiciary Subcommittee hearing on September 17, 2024, whistleblowers from
    leading AI companies, including OpenAI and Google, emphasized that AGI 
    is no longer a distant speculation but an impending reality. Helen Toner,
    a former OpenAI board member, testified that companies like OpenAI,
    Google, and Anthropic are earnestly pursuing AGI development. Former
    OpenAI researcher William Saunders noted that these companies are 
    investing billions toward this goal. Senators Josh Hawley and 
    Richard Blumenthal expressed concern over the lack of regulatory 
    oversight, warning of potential risks such as cyberattacks, 
    economic upheaval, and even human extinction. The article 
    calls for immediate regulatory action, public engagement, 
    and the implementation of transparency and security mandates 
    to manage the societal impacts of AGI responsibly.""",
}

# Ensure body_text is a string
body_text = str(body_text)

# First create a LangChain Document from the extracted text
document = LangChainDocument(page_content=body_text, metadata=metadata)

#Define chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

#Split the document into chunks using the textsplitter
chunks = text_splitter.split_documents([document])

#Embed the chunks into Chroma vectorstore
vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory=persist_directory)


### Article #2 (Futurism)

In [70]:
import requests
from bs4 import BeautifulSoup

# Step 1: Define the URL and fetch HTML content
url = "https://futurism.com/the-byte/sam-altman-excited-agi-2025"
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Step 2: Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Locate the main article body
# Look for an element that contains the main article text
# You may need to adjust these selectors based on the HTML structure of the Time website

article_body = soup.find("div", id="incArticle")  # Adjust if needed
if not article_body:
    article_body = soup.find("article")  # Use <article> tag as a fallback

# Step 4: Remove unwanted elements, such as ads or related links
for unwanted in article_body.find_all(["aside", "footer", "nav", "script", "style", "figure"]):
    unwanted.decompose()  # This removes the unwanted tags and their content

# Step 5: Extract and clean the text
body_text = article_body.get_text()

In [ ]:
# Save the text as a word document to look at if you want.
""" from docx import Document

# Create a new Document
doc = Document()

# Add the content of body_text to the document
doc.add_paragraph(body_text)

# Save the document
doc.save("body_text.docx") """

In [73]:
from langchain.schema import Document as LangChainDocument

# Now we need to chunk the text into smaller parts and add metadata

# Define the metadata
metadata = {
    "type": "article",
    "title": "SAM ALTMAN SAYS THE MAIN THING HE’S EXCITED ABOUT NEXT YEAR IS ACHIEVING AGI",
    "source": "Futurism",
    "author(s)": "Noor Al-Sibai",
    "date": "November 11 2024",
    "summary": """Article talks about how Sam Altman predicts there will be AGI in 2025.
    It is not specified exactly how that will be achieved, or where in the progress
    they (OpenAI) are.""",
}

# Ensure body_text is a string
body_text = str(body_text)

# First create a LangChain Document from the extracted text
document = LangChainDocument(page_content=body_text, metadata=metadata)

#Define chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

#Split the document into chunks using the textsplitter
chunks = text_splitter.split_documents([document])

#Embed the chunks into Chroma vectorstore
vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory=persist_directory)

### Article #3 (FUCK OFF)

In [ ]:
import re
from bs4 import BeautifulSoup

#Get the URL of the article
#url = "https://www.artificialintelligence-news.com/news/asi-alliance-launches-airis-learns-minecraft/"
#loader = WebBaseLoader([url])
#document = loader.load()  # Get the first document

html_content = document[0].page_content 
soup = BeautifulSoup(html_content, "html.parser")




'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nASI Alliance launches AIRIS that ‘learns’ in Minecraft\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n \n \n\n\n \n \n\n\n\n\n\n \n \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n                    ARTICLE\n                \n \n\nLOG IN\n\n \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNews\nCategories\n\nApplications\n\nChatbots\nFace Recognition\nVirtual Assistants\nVoice Recognition\n\n\nCompanies\n\nAmazon\nApple\nGoogle\nMeta (Facebook)\nMicrosoft\nNVIDIA\n\n\nDeep & Reinforcement Learning\nEnterprise\nEthics & Society\nIndustries\n\nEnergy\nEntertainment & Retail\nGaming\nHealthcare\nManufacturing\n\n\nLegislation & Government\nMachine Learning\nPrivacy\nResearch\nRobotics\nSecurity\nSurveillance\n\n\nEvents\nWebinars & Resources\nWork W

#### Using newspaper 3k

In [ ]:
#%pip install newspaper3k
#%pip install lxml_html_clean 
import nltk
from newspaper import Article

# Download NLTK data if not already downloaded
nltk.data.path.append("C:/Users/cleme/AppData/Roaming/nltk_data") 
nltk.download('punkt')



# Step 1: Define the URL
url = "https://www.artificialintelligence-news.com/news/asi-alliance-launches-airis-learns-minecraft/"

# Step 2: Create an Article object and download content
article = Article(url)
article.download()
article.parse()

# Step 3: Access the article's text and metadata
article_text = article.text
article_title = article.title

# Step 4: Split the text into sentences
sentences = nltk.sent_tokenize(article_text)
formatted_text = "\n".join(sentences)


In [10]:

# Step 5: Print the formatted article text
print("Title:", article_title)
print("\nFormatted Article Text:\n", article_text[:3000], "...")

Title: ASI Alliance launches AIRIS that ‘learns’ in Minecraft

Formatted Article Text:
 Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry) or Mastodon (@gadgetry@techhub.social)

The ASI Alliance has introduced AIRIS (Autonomous Intelligent Reinforcement Inferred Symbolism) that “learns” within the popular game, Minecraft.

AIRIS represents the first proto-AGI (Artificial General Intelligence) to harness a comprehensive tech stack across the alliance.

SingularityNET, founded by renowned AI researcher Dr Ben Goertzel, uses agent technology from Fetch.ai, incorporates Ocean Data for long-term memory 

### Article #4 (BBC)

In [67]:
import requests
from bs4 import BeautifulSoup

# Step 1: Define the URL and fetch HTML content
url = "https://www.bbc.com/news/articles/cj9jwyr3kgeo"


response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Step 2: Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Step 3: Locate the main article body
# Look for an element that contains the main article text
# You may need to adjust these selectors based on the HTML structure of the Time website

article_body = soup.find("div", id="main-content")  # Adjust if needed
if not article_body:
    article_body = soup.find("article")  # Use <article> tag as a fallback

# Step 4: Remove unwanted elements, such as ads or related links
for unwanted in article_body.find_all(["aside", "footer", "nav", "script", "style", "figure"]):
    unwanted.decompose()  # This removes the unwanted tags and their content

# Step 5: Extract and clean the text
body_text = article_body.get_text()

In [74]:
# Now we need to chunk the text into smaller parts and add metadata

# Define the metadata
metadata = {
    "type": "article",
    "title": "California governor blocks landmark AI safety bill",
    "source": "BBC",
    "author(s)": "Joao Da Silva",
    "date": "September 30 2024",
    "summary": """This article discusses California Governor Gavin Newsom’s veto of a significant AI safety bill that aimed to introduce some of the first AI regulations in the United States. The proposed legislation, authored by Senator Scott Wiener, would have mandated safety testing for advanced AI models, required a "kill switch" for isolating AI systems if they posed a threat, and enforced oversight on "Frontier Models" — the most advanced AI systems. Newsom argued the bill could hinder innovation, affecting even basic AI functions, and possibly push developers out of California. Instead, Newsom has called on experts to help create safeguards for AI while signing other bills to combat misinformation and deepfakes. The decision has left AI firms largely unregulated in California, despite concerns from lawmakers about the risks posed by unchecked AI development.""",
}

# Ensure body_text is a string
body_text = str(body_text)

# First create a LangChain Document from the extracted text
document = LangChainDocument(page_content=body_text, metadata=metadata)

#Define chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

#Split the document into chunks using the textsplitter
chunks = text_splitter.split_documents([document])

#Embed the chunks into Chroma vectorstore
vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=OpenAIEmbeddings(),
                                    persist_directory=persist_directory)

### Test Vector Store

In [23]:
# Generate embedding function (takes text and generates embedding using OpenAI' embedding model)
def generate_embedding(text):
    # The new API structure for embedding might look something like this
    # Replace `"text-embedding-ada-002"` with the correct model ID if it has changed
    response = openai.embeddings.create(input=[text], model="text-embedding-ada-002")
    embedding = response['data'][0]['embedding']
    return embedding

In [24]:
# Sample query for retrieval (adjust the query based on your use case)
sample_query = "Yampolskiy's research on AI safety"

# Retrieve documents from Chroma based on the sample query
retrieved_docs = vectorstore.similarity_search(sample_query, k=5)  # Retrieve top 5 matches

# Inspect the retrieved documents and their metadata
print("Retrieved Documents and Metadata:")
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocument {i + 1}:")
    print("Content:", doc.page_content[:400], "...")  # Show first 200 characters for context
    print("Metadata:", doc.metadata)  # Print all metadata fields
    print("-" * 50)

Retrieved Documents and Metadata:

Document 1:
Content: Roman Yampolskiy: So the problem of controlling AGI or superintelligence, in my opinion, is like the problem of creating a perpetual safety machine by analogy with a perpetual motion machine. It's impossible. Yeah, we may succeed and do a good job with GPT-5, 6, 7, but they just keep improving, learning, eventually self-modifying, interacting with the environment, interacting with malevolent actor ...
Metadata: {'source': 'Interview Transcript', 'speaker': 'Roman Yampolskiy', 'speaker_bio': "Roman Yampolskiy is a Latvian computer scientist specializing in AI safety and cybersecurity, holding a Ph.D. from the University at Buffalo, and currently directing the Cyber Security Lab at the University of Louisville's Speed School of Engineering.", 'speaker_role': 'Guest'}
--------------------------------------------------

Document 2:
Content: Roman Yampolskiy: So the problem of controlling AGI or superintelligence, in my opinion, is lik

## Vectore Store search WITH Metadata filtering

In [25]:
import openai

# Assume 'vectorstore' is your Chroma vectorstore with documents already embedded using OpenAIEmbeddings
# Initialize OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")  # Specify the model if needed

# Define the query
query = "What are the main points from the interview?"

# Perform a similarity search with metadata filtering
# Ensure your Chroma instance is initialized with your documents and metadata
results = vectorstore.similarity_search(
    query,  # Use the query string for similarity search
    k=5,  # Number of results to return
    filter={"source": "Interview Transcript"}  # Filter to include only chunks from interviews
)

# Display the results
for i, result in enumerate(results):
    print(f"Result {i + 1}:")
    print("Content:", result.page_content[:200], "...")  # Show a snippet of the content
    print("Metadata:", result.metadata)
    print("-" * 50)

Result 1:
Content: Lex Fridman: So that's one of the possible outcomes. But what in general is the idea of the paper? It's looking at multiple agents that are human-AI, like a hybrid system where there's humans and AIs, ...
Metadata: {'source': 'Interview Transcript', 'speaker': 'Lex Fridman', 'speaker_bio': 'Lex Fridman is a Russian-American computer scientist and podcaster, known for his work in artificial intelligence and his podcast featuring interviews with prominent figures across various fields.', 'speaker_role': 'Interviewer'}
--------------------------------------------------
Result 2:
Content: Lex Fridman: So that's one of the possible outcomes. But what in general is the idea of the paper? It's looking at multiple agents that are human-AI, like a hybrid system where there's humans and AIs, ...
Metadata: {'source': 'Interview Transcript', 'speaker': 'Lex Fridman', 'speaker_bio': 'Lex Fridman is a Russian-American computer scientist and podcaster, known for his work in artific

# Summarize interview

In [26]:
from docx import Document
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Load the Word document and extract text
doc = Document("corrected_interview.docx")
document_text = "\n".join([para.text for para in doc.paragraphs])

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Create a prompt template for summarization
system_message = SystemMessagePromptTemplate.from_template("You are a helpful assistant that summarizes documents.")
human_message = HumanMessagePromptTemplate.from_template("Please summarize the following document in 3 sentences:\n\n{document_text}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Generate the summary
summary = llm(chat_prompt.format(document_text=document_text))

print("Summary:")
print(summary)

C:\Users\cleme\AppData\Local\Temp\ipykernel_17048\3169080763.py:19: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = llm(chat_prompt.format(document_text=document_text))


Summary:
content='In a discussion between Lex Fridman and Roman Yampolskiy, the potential existential risks posed by superintelligent AI are explored, particularly the challenges of controlling AI systems that may evolve beyond human oversight. Yampolskiy argues that while current AI systems have already demonstrated unintended behaviors and failures, the unpredictable nature of future superintelligent AI could lead to catastrophic outcomes for humanity, including suffering and loss of meaning in life. They also discuss the concept of Ikigai risk, where the rise of AI might lead to widespread technological unemployment, resulting in individuals losing purpose, and propose that creating personal virtual universes could be a potential solution to the value alignment problem inherent in AI systems.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 4327, 'total_tokens': 4467, 'completion_tokens_details': {'accepted_prediction

# Article First draft

In [27]:
# Define the retriever to get the documents from the vectorstore
retriever = vectorstore.as_retriever(
    search_type = "mmr",  # Use Maximal Marginal Relevance (MMR) for search
    search_kwargs={"k": 5,
                   "lambda_mult": 0.7,
                   "fetch_k": 10})

#Define the prompt which will be used by the RAG
#This takes the prompts as strings, and converts them into the format the model expects.
def create_prompt(system_prompt, human_prompt):
    system_message = SystemMessagePromptTemplate.from_template(system_prompt)
    human_message = HumanMessagePromptTemplate.from_template(human_prompt)
    return ChatPromptTemplate.from_messages([system_message, human_message])

# Customize the system and human prompts
system_prompt = """You are an AI assistant to a journalist.
Your goal is to generate accurate, concise answers using only the provided context.
Do not make assumptions or add information that is not explicitly present in the context.

When referencing information, quote the source directly using quotation marks and preface each quote with an attribution (e.g., "According to [source],..."). Only use exact phrases from the context when quoting to ensure accuracy.

If quoting is not possible, you may paraphrase or summarize carefully, but avoid introducing any new interpretation. Aim to maintain the original meaning of the content as closely as possible.

Here is the context: {context}
"""

# Customize the human prompt
human_prompt = "{question}"

#Put them together using the create_prompt function
prompt_template = create_prompt(system_prompt, human_prompt)

#Define a function that takes all the documents fetched by the receiver and formats 
#them into a single string, with double spaces between each document.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Set up the RAG chain
#1. First the receiver takes the question and fetches the relevant dokuments from the vector store
#2. The question is passed through to the next step of the chain
#3. The question and context (documents) are passed to the prompt template
#4. Prompt template is sent to the LLM
#5. The output from the LLM is parsed into a string
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm  # ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

#Define the question and invoke the chain.
question = """Write an article of approximately 500 words on artificial general intelligence (AGI).
Include the current state of development, key risks associated with AGI, and insights from leading experts in the field.
Use direct quotes from the provided context to support key points, and attribute each quote to the respective source."""

response = rag_chain.invoke(question)
print(response)




**Artificial General Intelligence: Current Developments, Risks, and Expert Insights**

Artificial General Intelligence (AGI) represents a significant leap in the field of artificial intelligence (AI), promising capabilities that could exceed human intelligence across a wide array of tasks. Unlike current Artificial Narrow Intelligence (ANI) systems, which are designed for specific tasks such as facial recognition or voice assistance, AGI has the potential to learn and perform any cognitive task that a human can. Recent discussions among experts highlight both the advancements toward AGI and the substantial risks associated with its development.

The current state of AGI development is characterized by rapid progress, with leading AI companies treating the creation of AGI as a serious endeavor. Helen Toner, a former board member of OpenAI, emphasized this urgency during a Senate Judiciary Subcommittee hearing, stating, “The biggest disconnect that I see between AI insider perspectives a

In [28]:
# Save response as a word file
with open("article.docx", "w") as f:
    f.write(response)

In [29]:
# See what data the response was based on

## To understand what information is being used to answer the question, we can 
# retrieve the chunks that are being used. This makes it possible to analyze the output.

# Retrieve the chunks for the given question
retrieved_docs = retriever.invoke(question)

# Define the file path where you want to save the output
file_path = "documents_and_metadata_output.txt"

# Open the file in write mode
with open(file_path, "w", encoding="utf-8") as file:
    # Write a header for the output
    file.write("Documents and Metadata Provided as Context:\n\n")
    
    # Loop through each document and write its content and metadata to the file
    for i, doc in enumerate(retrieved_docs):
        file.write(f"Document {i + 1}\n")
        file.write("Content:\n")
        file.write(doc.page_content + "\n\n")  # Writing the full content
        
        file.write("Metadata:\n")
        for key, value in doc.metadata.items():
            file.write(f"{key}: {value}\n")
        
        file.write("-" * 50 + "\n\n")

print(f"Output successfully saved to {file_path}")

Output successfully saved to documents_and_metadata_output.txt
